# Ratings

In [1]:
import pandas as pd

In [20]:
catcher_projections_df = pd.read_csv('../data/projections/positions/catcher.csv')
first_base_projections_df = pd.read_csv('../data/projections/positions/first-base.csv')
second_base_projections_df = pd.read_csv('../data/projections/positions/second-base.csv')
third_base_projections_df = pd.read_csv('../data/projections/positions/third-base.csv')
shortstop_projections_df = pd.read_csv('../data/projections/positions/shortstop.csv')
outfield_projections_df = pd.read_csv('../data/projections/positions/outfield.csv')
designated_hitter_projections_df = pd.read_csv('../data/projections/positions/designated-hitter.csv')
starter_projections_df = pd.read_csv('../data/projections/positions/starter.csv')
reliever_projections_df = pd.read_csv('../data/projections/positions/reliever.csv')

In [21]:
points_lookup = {
    'batting': {
        'R': 1,
        'TB': 1,
        'RBI': 1,
        'BB': 1,
        'K': -1,
        'SB': 1
    },
    'pitching': {
        'IP': 3,
        'H': -1,
        'ER': -2,
        'HD': 2,
        'BB': -1,
        'K': 1,
        'W': 2,
        'L': -2,
        'SV': 5
    }
}

In [22]:
def calculate_batter_fantasy_points(df):
    # Create a copy with just needed columns
    result_df = df[['Name', 'Team']].copy()
    
    # Calculate total bases
    total_bases = (df['1B'] + 
                  (2 * df['2B']) + 
                  (3 * df['3B']) + 
                  (4 * df['HR']))

    # Calculate fantasy points
    fantasy_points = (
        points_lookup['batting']['R'] * df['R'] +
        points_lookup['batting']['TB'] * total_bases + 
        points_lookup['batting']['RBI'] * df['RBI'] +
        points_lookup['batting']['BB'] * df['BB'] +
        points_lookup['batting']['K'] * df['SO'] +
        points_lookup['batting']['SB'] * df['SB']
    )

    # Add calculated columns to new dataframe
    result_df = result_df.assign(
        FPTS=fantasy_points,
    )
    
    return result_df

# Calculate fantasy points for each position
catcher_fpoints_df = calculate_batter_fantasy_points(catcher_projections_df)
first_base_fpoints_df = calculate_batter_fantasy_points(first_base_projections_df)
second_base_fpoints_df = calculate_batter_fantasy_points(second_base_projections_df)
third_base_fpoints_df = calculate_batter_fantasy_points(third_base_projections_df)
shortstop_fpoints_df = calculate_batter_fantasy_points(shortstop_projections_df)
outfield_fpoints_df = calculate_batter_fantasy_points(outfield_projections_df)
designated_hitter_fpoints_df = calculate_batter_fantasy_points(designated_hitter_projections_df)

In [23]:
def calculate_pitcher_fantasy_points(df):
    # Create a copy with just needed columns
    result_df = df[['Name', 'Team']].copy()
    
    # Calculate fantasy points
    fantasy_points = (
        points_lookup['pitching']['IP'] * df['IP'] +
        points_lookup['pitching']['H'] * df['H'] +
        points_lookup['pitching']['ER'] * df['ER'] +
        points_lookup['pitching']['BB'] * df['BB'] +
        points_lookup['pitching']['K'] * df['SO'] +
        points_lookup['pitching']['W'] * df['W'] +
        points_lookup['pitching']['L'] * df['L'] +
        points_lookup['pitching']['SV'] * df['SV']
    )

    # Add calculated columns to new dataframe 
    result_df = result_df.assign(
        FPTS=fantasy_points,
    )
    
    return result_df

# Calculate fantasy points for pitchers
pitcher_fpoints_df = pd.concat([
    calculate_pitcher_fantasy_points(starter_projections_df),
    calculate_pitcher_fantasy_points(reliever_projections_df)
]).reset_index(drop=True)

In [24]:
def calculate_vorp(position_df, replacement_rank):
    # Sort by fantasy points in descending order
    sorted_df = position_df.sort_values('FPTS', ascending=False).reset_index(drop=True)
    
    # Get replacement level points (player at replacement_rank)
    if len(sorted_df) >= replacement_rank:
        replacement_points = sorted_df.iloc[replacement_rank-1]['FPTS']
    else:
        replacement_points = sorted_df.iloc[-1]['FPTS']
    
    # Calculate VORP for all players
    sorted_df['VORP'] = sorted_df['FPTS'] - replacement_points
    
    return sorted_df

# Calculate VORP for each position using appropriate replacement ranks
catcher_fpts_vorp_df = calculate_vorp(catcher_fpoints_df, 12)
first_base_fpts_vorp_df = calculate_vorp(first_base_fpoints_df, 15)
second_base_fpts_vorp_df = calculate_vorp(second_base_fpoints_df, 15)
third_base_fpts_vorp_df = calculate_vorp(third_base_fpoints_df, 15)
shortstop_fpts_vorp_df = calculate_vorp(shortstop_fpoints_df, 15)
outfield_fpts_vorp_df = calculate_vorp(outfield_fpoints_df, 40)
designated_hitter_fpts_vorp_df = calculate_vorp(designated_hitter_fpoints_df, 15)
pitcher_fpts_vorp_df = calculate_vorp(pitcher_fpoints_df, 70)

In [25]:
# Combine all position dataframes into one unified dataframe
all_positions_df = pd.concat([
    catcher_fpts_vorp_df.assign(Position='C'),
    first_base_fpts_vorp_df.assign(Position='1B'),
    second_base_fpts_vorp_df.assign(Position='2B'),
    third_base_fpts_vorp_df.assign(Position='3B'),
    shortstop_fpts_vorp_df.assign(Position='SS'),
    outfield_fpts_vorp_df.assign(Position='OF'),
    designated_hitter_fpts_vorp_df.assign(Position='DH'),
    pitcher_fpts_vorp_df.assign(Position='P')
]).reset_index(drop=True)

# Create sorted dataframes by FPTS and VORP
all_positions_fpts_sorted_df = all_positions_df.sort_values('FPTS', ascending=False).reset_index(drop=True)

# Save to CSV
all_positions_fpts_sorted_df.to_csv('../data/generated/all-positions-fpts.csv', index=False)